In [ ]:
import os
import torch

# Импорт модели
from Model import SiameseViT

# Импорт функций подготовки данных
from Data_process import Data_Transform

# Импорт цикла обучения и валидации 
from Trainer import train_model, evaluate_model

# Импорт функций визуализации
from Visual_func import plot_training_history, plot_confusion_matrix, visualize_pair

# Импорт функции потерь и функций расчета метрик
from Loss_Metrics import AdaptiveContrastiveLoss, calculate_metrics, create_confusion_matrix



# Проверка наличия GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Устройство: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Всего GPU памяти: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Подготовка данных
### Путь к данным

In [ ]:
Data_PATH = r"/Data"

### Обработка путей

In [ ]:
Images_path = os.path.join(Data_PATH, "full")
Data_csv_path = os.path.join(Data_PATH, "data.csv")
Test_data_path = os.path.join(Data_PATH, "test_data.csv")
Train_data_path = os.path.join(Data_PATH, "train_data.csv")

print(f"Корневая директория: {Data_PATH}")
print(f"Изображения: {Images_path}")
print(f"Train CSV: {Train_data_path}")
print(f"Test CSV: {Test_data_path}")

## Создание обучающих и валидационных данных
### Параметры данных

In [ ]:
BATCH_SIZE = 512
BATCH_SIZE_VAL = 256

image_size = (128, 256)

### Создаем и загружаем датасеты

In [ ]:
data_transformer = Data_Transform(Train_data_path, Test_data_path, Images_path, image_size = image_size)
train_ds, _ = data_transformer.create_datasets(vizual_id=0)

### Пример обучающих данных

In [ ]:
for i in range(10, 16):
    visualize_pair(train_ds, i)

### Создаем даталоадеры

In [ ]:
train_loader, test_loader = data_transformer.create_dataloaders(BATCH_SIZE, BATCH_SIZE_VAL)

## Создание модели

In [ ]:
# Инициализация модели
model_params = {
    "feature_dim": 1024,
    "embed_dim": 256,
    "img_size": image_size
}

model = SiameseViT(**model_params).to(device)
print(f"Общее количество параметров модели: {sum(p.numel() for p in model.parameters()):,}")

## Инициализация параметров обучения

In [ ]:
# Количнство эпох
warmup_epochs = 3
total_epochs = 20

# Пути сохранения моделей
best_model_path = 'best_model.pth'
final_model_path = 'final_model.pth'


# Оптимизатор с разделением весов
backbone_params = list(model.feature_extractor.parameters())
head_params = list(model.asymmetric_comparator.parameters())

optimizer = torch.optim.AdamW([
    {'params': backbone_params, 'lr': 1e-4},
    {'params': head_params, 'lr': 3e-4}
], weight_decay=1e-4)

# Планировщики обучения
warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer, start_factor=0.1, total_iters=warmup_epochs
)
main_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=total_epochs - warmup_epochs, eta_min=1e-6
)

# Функция потерь
criterion = AdaptiveContrastiveLoss(margin=0.5, alpha=0.8)

## Обучние модели

In [ ]:
trained_model, history = train_model( model, total_epochs, train_loader, test_loader,
                            warmup_scheduler, main_scheduler, warmup_epochs,
                            optimizer, criterion, best_model_path, final_model_path, device,
                            plot_on_train = True )

### Итоговые графики обучения

In [ ]:
plot_training_history(history)

### Финальная оценка

In [ ]:
model = SiameseViT(**model_params).to(device)
model.load_state_dict(torch.load(best_model_path)['model_state_dict'])
print("BEST MODEL:")
evaluate_model(model, test_loader, device)

model = SiameseViT(**model_params).to(device)
model.load_state_dict(torch.load(final_model_path)['model_state_dict'])
print("\nFINAL MODEL:")
evaluate_model(model, test_loader, device)